# Background

The simulations record the trajectories (a series of x,y,z coordinates, with associated amplitude of each point) of products resulting from collision/reaction between Mg22 and alpha particles in ATTPC.

# User-Desired Settings

The isotope used in this experiment is Mg22.

In [ ]:
ISOTOPE = 'Fission'

The neural network model requires a fixed number of inputs. Whereas the actual events comprise different number of points, we will select exactly 512 points (may be redundant) as final inputs of each event.

In [ ]:
sample_size = 512

We create a folder named "test" to store the outputs.

In [ ]:
dir_name = 'fission_data/'

# Data Processing

## Import Libraries

In [ ]:
import h5py
import numpy as np
import tqdm
import math
import random
import copy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits import mplot3d

## Import Data

In [ ]:
def import_data(file):
    event_ids = list(file.keys())
    num_of_event = len(event_ids)
    ev_lens = np.zeros(num_of_event, int)
    for i in range(num_of_event):
        event_id = event_ids[i]
        event = file[event_id]['HitArray']
        ev_lens[i] = len(file[event_id]['HitArray'])
    evlen_path = dir_name + ISOTOPE + '_sim' + '_XYZAPPE_ev_lens'
    np.save(evlen_path, ev_lens)

    data = np.zeros((num_of_event, np.max(ev_lens), 7), float) # XYZAPPE
    for n in tqdm.tqdm(range(num_of_event)):
        event_id = event_ids[n]
        event = file[event_id]['HitArray']
        #converting event into an array
        for i,e in enumerate(event):
            instant = np.array(list(e))
            data[n][i][0:3] = np.array(instant[0:3]) # x,y,z
            data[n][i][3] = np.array(instant[4]) # amplitude
            data[n][i][4] = np.array(instant[5])-1 # particleID--lower index to start at 0
            data[n][i][5] = np.arange(1,np.max(ev_lens)+1)[i] # pointID
            data[n][i][-1] = float(n) # eventID
    data_path = dir_name  + ISOTOPE + '_sim' + '_XYZAPPE'
    np.save(data_path, data)
    
    return ev_lens, data

In [ ]:
file = h5py.File(dir_name + 'Bi200SimMoreTriggered.h5', 'r')

ev_lens, data = import_data(file)

When running this notebook the second time, simply reload the data (instead of spending 10 min to repeat the step above).

In [ ]:
ev_lens = np.load(dir_name + ISOTOPE + '_sim_XYZAPPE_ev_lens.npy')
data = np.load(dir_name + ISOTOPE + '_sim_XYZAPPE.npy')

### Plot Events

In [ ]:
def visualize(arr, r, c):
    fig = plt.figure(figsize=(17,r*4))

    for i in range(r*c):
        
        ax = fig.add_subplot(r, c, i+1, projection='3d') # Creating subplots
        cloud = arr[i]
        # Splitting 4D point cloud into x, y, z coordinates and color
        xs = cloud[:, 0]
        ys = cloud[:, 1]
        zs = cloud[:, 2]
        colors = cloud[:, 3]

        sc = ax.scatter(xs, ys, zs, c=colors, cmap=plt.cool(), s = 1)
        ax.set_title(f'Point Cloud {i+1}')
        plt.colorbar(sc, shrink = 0.5)
    plt.show()

## Sample Points

In [ ]:
evlen_path = dir_name + ISOTOPE + '_sim_XYZAPPE_ev_lens.npy'
data_path = dir_name + ISOTOPE + '_sim_XYZAPPE.npy'
data_noNull = np.load(data_path)
num_of_event = len(data_noNull)
max_ev_len = len(data_noNull[0])
ev_lens = np.load(evlen_path)
data_sampled = np.zeros((num_of_event, sample_size, 7), float) #XYZAPPE

for n in tqdm.tqdm(range(num_of_event)):
    ev_len = ev_lens[n]
    if ev_len >= sample_size:
        data_sampled[n,:sample_size,:] = data_noNull[n,:sample_size,:]
    else:
        data_sampled[n,:ev_len,:] = data_noNull[n,:ev_len,:]
        need = sample_size - ev_len
        random_points = np.random.choice(range(ev_len), need, replace=True if need > ev_len else False) 
        instant = ev_len
        for r in random_points:
            data_sampled[n,instant,:] = data_noNull[n,r,:] 
            instant += 1

data_path = dir_name + ISOTOPE + '_sim_sampled_size' + str(sample_size)
np.save(data_path, data_sampled)

## Get XYZC

In [ ]:
data = np.load(dir_name + ISOTOPE + '_sim_sampled_size' + str(sample_size) + '.npy')
new_data = data[:,:, [0,1,2,3]]
data_path = dir_name + ISOTOPE + '_sim_sampled_XYZC'
np.save(data_path, new_data)

## Normalize

In [ ]:
data_path = dir_name + ISOTOPE + '_sim_sampled_XYZC.npy'
data_sampled = np.load(data_path)
data_scaled = data_sampled

data_sampled[:,:,3] = np.where(data_sampled[:,:,3] > 0, data_sampled[:,:,3], 1)
data_sampled[:,:,3] = np.where(data_sampled[:,:,3] < 10000, data_sampled[:,:,3], 10000)   
data_scaled[:,:,3] = np.log10(data_sampled[:,:,3])

for n in range(3):
    if n == 0 or n == 1:
        data_scaled[:,:,n] /= 250
    else:
        data_scaled[:,:,n] = data_scaled[:,:,n]/500 + 1
    # data_sampled[:,:,n] = np.where(data_sampled[:,:,n] < 1000, data_sampled[:,:,n], 1000)
    # data_sampled[:,:,n] = np.where(data_sampled[:,:,n] > -1000, data_sampled[:,:,n], -1000)
    # mean = np.mean(data_sampled[:,:,n])
    # std = np.std(data_sampled[:,:,n])
    # data_scaled[:,:,n] = (data_sampled[:,:,n] - mean) / std

data_path = dir_name + ISOTOPE + '_sim_sampled_scaled_XYZC'
np.save(data_path, data_scaled)

In [ ]:
data_path = 'fission_data/Fission_sim_XYZAPPE_ev_lens.npy'
data = np.load(data_path)
print(np.mean(data))

In [ ]:
import h5py
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

file_path = dir_name + 'Bi200SimTriggered.h5'

def plot_event(event_num):
    with h5py.File(file_path, 'r') as file:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        
        # Iterate through each event group in the file
        event_key = f'Event_[{event_num}]'
        
        event_group = file[event_key]
        
        # Assuming 'HitArray' is the dataset name containing the hits
        # Adjust the dataset name and indexing as per your data structure
        if 'HitArray' in event_group:
            hits = event_group['HitArray'][:]
            print(f"{event_key}: {len(hits)}")           
            x = [hit[0] for hit in hits]
            y = [hit[1] for hit in hits]
            z = [hit[2] for hit in hits]
                        
            ax.scatter(x, y, z, label=event_key)
        
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        # ax.view_init(90,-90)
        plt.show()

In [ ]:
data_path ='fission_data/Fission_sim_sampled_size512.npy'
data_sampled = np.load(data_path)
visualize(data_sampled, 20, 2)